Quick analysis

In [1]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

In [2]:
import pygama

In [3]:
dg = DataGroup('cage.json', load=True)
run = 110
str_query = f'run=={run} and skip==False'
dg.fileDB.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.fileDB[view_cols])

Loading file key list from: ./fileDB.h5
     runtype  run  cycle     startTime    runtime  threshold
1182     alp  110   1184  1.608244e+09  23.309349       16.0
1183     alp  110   1185  1.608245e+09  29.997711       16.0
1184     alp  110   1186  1.608247e+09  29.988346       16.0
1185     alp  110   1187  1.608249e+09  29.977714       16.0
1186     alp  110   1188  1.608251e+09  29.985387       16.0
1187     alp  110   1189  1.608252e+09  29.992253       16.0
1188     alp  110   1190  1.608254e+09  15.499158       16.0


In [4]:
#get runtime, startime, runtype

runtype_list = np.array(dg.fileDB['runtype'])
runtype = runtype_list[0]

rt_min = dg.fileDB['runtime'].sum()
u_start = dg.fileDB.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [5]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['angle'])[0]
    print(f'Radius: {radius}; Angle: {angle}')
# else:

Radius: 10.0; Angle: -180.0


In [6]:
lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
hit_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']

print(f'lh5_dir: {lh5_dir}')
print(f'hit list: {hit_list}')

lh5_dir: /global/project/projectdirs/legend/users/gothman/CAGE/
hit list: 1182    /global/project/projectdirs/legend/users/gothm...
1183    /global/project/projectdirs/legend/users/gothm...
1184    /global/project/projectdirs/legend/users/gothm...
1185    /global/project/projectdirs/legend/users/gothm...
1186    /global/project/projectdirs/legend/users/gothm...
1187    /global/project/projectdirs/legend/users/gothm...
1188    /global/project/projectdirs/legend/users/gothm...
dtype: object


In [7]:
cal = False
# cal = True

In [8]:
if cal == True:
    df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'trapEmax_cal', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
else:
    df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    

loading data for /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1184_hit.lh5 /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1185_hit.lh5 /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1186_hit.lh5 /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1187_hit.lh5 /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1188_hit.lh5 /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1189_hit.lh5 /global/project/projectdirs/legend/users/gothman/CAGE//hit/cage_run110_cyc1190_hit.lh5


In [9]:
# lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
# print(lh5_dir)
# hit_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
# print(hit_list)
# df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'trapEmax_cal', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# # df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# # print(df_hit)
# print(df_hit.columns)

# dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)

In [10]:
# get raw files for wfs

raw_lh5_dir = dg.lh5_dir
raw_list = raw_lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']

print(raw_list)

# don't actually need this, just file list
# raw_df = lh5.load_dfs(raw_list, ['energy'], 'ORSIS3302DecoderForEnergy/raw')



1182    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1183    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1184    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1185    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1186    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1187    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1188    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
dtype: object


In [11]:
# use baseline cut
df_cut = df_hit.query('bl > 8500 and bl < 10000').copy()

# add new A/E column
# df_cut['aoe'] = df_cut['A_10'] / df_cut['trapEmax']

#create dcr/E 
df_cut['dcr_norm'] = df_cut['dcr_raw'] / df_cut['trapEmax']

#create 10-90
df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

#create 50-90
df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

In [12]:
#creat new DCR
const = 0.0555
df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

In [13]:
# baseline cut
bl = df_hit['bl']
bl_cut = (bl > 8500) & (bl < 10000)


In [14]:
# energy cut 

if cal == True:
    energy = df_hit['trapEmax_cal']
    energy_cut = (energy > 1450) & (energy < 1470) # 1460 peak
    
else:
    energy = df_hit['trapEmax']
    energy_cut = (energy > 3597) & (energy < 3617) # 1460 peak, uncalibrated
    
# print(type(energy))
# print(energy_cut)
print(energy_cut.value_counts())

False    714978
True        349
Name: trapEmax, dtype: int64


In [15]:
cut = bl_cut & energy_cut

In [16]:
# for debugging

# print(cut)
print(cut.value_counts())

False    714978
True        349
dtype: int64


In [17]:
dsp_config_file = ('./config_dsp.json')
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz', 'wf_trap'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "Trap (max={trapEftp:.0f})"],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEftp'], # add hlines and vlines
                          x_lim=(0, 82500) # x axis range
                         )

KeyError: 0

In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]


In [ ]:
# this is with 6-4-6 trap

In [ ]:
browser.draw_next(5)

In [ ]:
df_hit['trapEftp'][483080]

In [ ]:
energy_cut = (trapEmax_cal>355) & (trapEmax_cal<360)
bl_cut = (bl > 8500) & (bl < 10000)
cut2 = energy_cut & bl_cut
dsp_config_file = ('./config_dsp.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz', 'wf_trap'], # names of waveforms from dsp config file
                          selection   = cut2,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "Trap (max={trapEmax:.0f})"],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEmax', 'tp_0'], # add hlines and vlines
                          x_lim=(15000, 60000) # x axis range
                         )

In [ ]:
browser2.draw_next()

In [ ]:
print(trapEmax_cal[615])
print(energy_cut[615])
# np.nonzero(browser.selection.values)
np.nonzero(energy_cut.values)

In [ ]:
energy_cut = (trapEmax_cal>355) & (trapEmax_cal<360)
bl_cut = (bl > 8500) & (bl < 10000)
cut2 = energy_cut & bl_cut
dsp_config_file = ('./config_dsp.json')
browser3 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub'], # names of waveforms from dsp config file
                          selection   = cut2,           # Apply cut
                          #wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          #legend=['Waveform'],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEmax', 'tp_0'], # add hlines and vlines
                          x_lim=(15000, 60000) # x axis range
                         )

In [ ]:
browser3.draw_next()

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 1000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEmax_cal'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
# plt.title(f'trapEmax: {runtype} run {run}; {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 0, 6000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(df_cut['trapEmax_cal'], df_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
dlo, dhi, dpb = -0.1, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()

In [ ]:
old_dcr_cut = df_cut.query('trapEmax > 5000 and trapEmax < 15000 and dcr_norm > -0.0515 and dcr_norm < -0.02').copy()
len(old_dcr_cut)

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
e_cut = df_hit.query(f' trapEmax_cal > 348 and trapEmax_cal < 356').copy()
#creat new DCR
const = 0.0555
e_cut['dcr_linoff'] = df_hit['dcr_raw'] + const*df_hit['trapEmax']

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 325, 375, 0.5
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(e_cut['trapEmax_cal'], e_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax_cal', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E 351keV peak: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
# etype = 'trapEmax'

elo, ehi, epb = 325, 375, 0.5
# elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(e_cut['trapEmax_cal'], e_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR 351 keV peak: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
cut2 = (df_cut['dcr_linoff']>50) & (df_cut['dcr_linoff']<200) & (df_cut['trapEmax']>5000) & (df_cut['trapEmax']<12000)
# cut2 = (df_cut['trapEmax']>5000) & (df_cut['trapEmax']<10000)
cut = (df_cut['trapEmax_cal']>1450) & (df_cut['trapEmax_cal']<1470)

In [ ]:
# Now set up browsing waveforms
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                          waveforms='wf_pz', # baseline subtracted waveform
                          #legend=['trapEmax', 'AoE'], # build a legend
                          align='tp_0', # align waveforms along t0
                          x_lim=(-500, 80000), # x axis range
                          #norm='trapEmax', # normalize by energy
                          selection = cut, # use our mask from before to draw MS wfs
                          n_drawn = 8, # Draw 8 at a time
                         )

In [ ]:
# Now set up browsing waveforms
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                          waveforms='wf_pz', # baseline subtracted waveform
                          #legend=['trapEmax', 'AoE'], # build a legend
                          align='tp_0', # align waveforms along t0
                          x_lim=(-500, 80000), # x axis range
                          #norm='trapEmax', # normalize by energy
                          selection = cut2, # use our mask from before to draw MS wfs
                          n_drawn = 8, # Draw 8 at a time
                         )

In [ ]:
# Draw first waveform in file
browser.draw_entry(1)

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
# browser.draw_next()
# browser2.ax = browser.ax
# browser2.fig = browser.fig
# browser2.draw_next()

In [ ]:
# Draw first waveform in file
browser2.draw_entry(5)

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
# browser.draw_next()
# browser2.ax = browser.ax
# browser2.fig = browser.fig
# browser2.draw_next()